In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [2]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT'): # Change folder
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)

nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+str(nameFileS))
print('pathFileS:'+str(pathFileS))
print('rootFileS:'+str(rootFileS))

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

nameFileS:0    parameters.json
1    parameters.json
2    parameters.json
3    parameters.json
4    parameters.json
5    parameters.json
6    parameters.json
7    parameters.json
8    parameters.json
9    parameters.json
dtype: object
pathFileS:0    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/binary_search/p...
1    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/floydwarshall/p...
2    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/bfs/parameters....
3    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/banker_algorith...
4    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/bellmanford/par...
5    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/selectionsort/p...
6    ../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/kruskal/paramet...
7    ../../../SLIDE-x

In [3]:
# .json file loop
#df = pd.DataFrame(columns=['Function','Device','Data_Type',
#                           'Num_Sc','Range_Sc','Num_ScI','Range_ScI',
#                           'Num_Arr','Range_Arr'])

listD = ["Arm","Atmega328p","Leon3","RiscV","Thumb"] # Insert the device/ISS you want to analyze
# listD = ["Bambu"]
listDT = ["int8_t","int16_t","int32_t","int64_t"] # Insert the data type you want to analyze
# listDT = ["float","double"]

listrange = [0, 0]
listrangeI = [0, 0]
listdtI = [' ']
listdt = [' ']
listfunI = [' ']
listfun = [' ']
listdevI = [' ']
listdev = [' ']
listboardI = [' ']
listboard = [' ']

for i in range(0, len(pathFile)):
    pF = pathFile[i]
    print('pF:'+str(pF))

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        # spR = rootFile[i].split('\\')
        spR = rootFile[i].rsplit('\\', 1)
        print('spR:'+str(spR))
    else:
        # spR = rootFile[i].split('/')
        spR = rootFile[i].rsplit('/', 1)
        print('spR:'+str(spR))
    
    function = spR[-1]
    print('function:'+str(function))
    
    for j in range(0, len(listD)):
        board = listD[j]
        print('board:'+str(board))

        #Device loop
        deviceList = list(obj.keys())
        print('deviceList:'+str(deviceList))
        deviceListR = []
        for u in range(0, len(deviceList)):
            if deviceList[u] in listD:
                deviceListR.append(deviceList[u])
                print('deviceListR:'+str(deviceList[u]))

        for n in range(0, len(deviceListR)):
            # Data_type loop
            device = deviceListR[n]
            print('device:'+str(device))
            dtarr = list(obj[deviceListR[n]].keys())
            print('dtarr:'+str(dtarr))

            dtarrR = []
            for u in range(0, len(dtarr)):
                if dtarr[u] in listDT:
                    dtarrR.append(dtarr[u])
                    print(f'dtarr[{u}]:'+str(dtarr[u]))

            for j in range(0, len(dtarrR)):
                dataype = dtarrR[j]
                print('dataype:'+str(dtarrR[j]))
                lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
                print('lsubNP:'+str(lsubNP))
                lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))
                print('lsubVP:'+str(lsubVP))
                ltot = len(lsubNP)
                print('ltot:'+str(ltot))

                kl = []
                for k in range(0, ltot):
                    flag = 0
                    if not "[" in lsubNP[k]:
                        for keysl in lsubNP:
                            if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                                flag = 1
                                kl.append(k) 
                                print('kl:'+str(k))

                m = 0
                for item in lsubVP:
                    #print(function, lsubVP)
                    if ';' in item:
                        newstr = item.split(";")[0]
                        newstr = newstr.replace("[", "")
                        newstr = newstr.replace("]", "")
                        try:
                            el1 = float(newstr.split(",")[0])
                            el2 = float(newstr.split(",")[1])
                        except ValueError as e:
                            print(f"ValueError: could not convert string to float: {e}")
                            el1 = 0
                            el2 = 0
                        strint = el1
                        rangeint = el2 - el1
                        lr = np.array([strint, rangeint])
                        if m in kl:
                            listrangeI = np.vstack([listrangeI, lr])
                            listdtI.append(dataype)
                            listfunI.append(function)
                            # listboardI.append(board)
                            listdevI.append(device)
                        else:
                            listrange = np.vstack([listrange, lr])
                            listdt.append(dataype)
                            listfun.append(function)
                            # listboard.append(board)
                            listdev.append(device)
                    m = m + 1

pF:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/binary_search/parameters.json
spR:['../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT', 'binary_search']
function:binary_search
board:Arm
deviceList:['args', 'ALL', 'Arm', 'Thumb', 'Atmega328p', '8051', 'Leon3', 'RiscV', 'Bambu']
deviceListR:Arm
deviceListR:Thumb
deviceListR:Atmega328p
deviceListR:Leon3
deviceListR:RiscV
device:Arm
dtarr:['int8_t', 'uint8_t', 'int16_t', 'uint16_t', 'int32_t', 'uint32_t', 'int64_t', 'uint64_t', 'float', 'double']
dtarr[0]:int8_t
dtarr[2]:int16_t
dtarr[4]:int32_t
dtarr[6]:int64_t
dataype:int8_t
lsubNP:['n', 'arr[n]', 'key']
lsubVP:['[2,32];10', '[-128,127]', '[-128,127];10']
ltot:3
kl:0
dataype:int16_t
lsubNP:['n', 'arr[n]', 'key']
lsubVP:['[2,32];10', '[-32766,32766]', '[-32766,32766];10']
ltot:3
kl:0
dataype:int32_t
lsubNP:['n', 'arr[n]', 'key']
lsubVP:['[2,32];10', '[-2147483640,2147483640]', '[-2147483640,2147483640];10']
ltot:3
kl:0
dat

In [4]:
listrangeS = pd.DataFrame(listrange)
listdtS = pd.Series(listdt)
listfunS = pd.Series(listfun)
listdevS = pd.Series(listdev)
# listboardS = pd.Series(listboard)
dfS = pd.concat([listdevS, listfunS, listdtS, listrangeS], axis=1) # listboardS, 
dfS.columns = ['DEVICE', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE']
dfS = dfS.iloc[1:]

In [5]:
dfS

,DEVICE,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,Arm,binary_search,int8_t,-1.280000e+02,2.550000e+02
2,Arm,binary_search,int16_t,-3.276600e+04,6.553200e+04
3,Arm,binary_search,int32_t,-2.147484e+09,4.294967e+09
4,Arm,binary_search,int64_t,-9.223372e+18,1.844674e+19
5,Thumb,binary_search,int8_t,-1.280000e+02,2.550000e+02
...,...,...,...,...,...
396,Leon3,gcd,int16_t,2.000000e+00,3.276400e+04
397,Leon3,gcd,int32_t,2.000000e+00,2.147484e+09
398,Leon3,gcd,int32_t,2.000000e+00,2.147484e+09
399,Leon3,gcd,int64_t,2.000000e+00,9.223372e+18


In [6]:
listrangeIS = pd.DataFrame(listrangeI)
listdtIS = pd.Series(listdtI)
listfunIS = pd.Series(listfunI)
listdevIS = pd.Series(listdevI)
# listboardIS = pd.Series(listboardI)
dfSI = pd.concat([listdevIS, listfunIS, listdtIS, listrangeIS], axis=1) # listboardIS ,
dfSI.columns = ['DEVICE', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE'] # 'BOARD',
dfSI = dfSI.iloc[1:]

In [7]:
dfSI

,DEVICE,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,Arm,binary_search,int8_t,2.0,30.0
2,Arm,binary_search,int16_t,2.0,30.0
3,Arm,binary_search,int32_t,2.0,30.0
4,Arm,binary_search,int64_t,2.0,30.0
5,Thumb,binary_search,int8_t,2.0,30.0
...,...,...,...,...,...
896,Leon3,astar,int64_t,2.0,30.0
897,RiscV,astar,int8_t,2.0,30.0
898,RiscV,astar,int16_t,2.0,30.0
899,RiscV,astar,int32_t,2.0,30.0


In [8]:
serS = dfS.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serS)):
    arrlimt = []
    ml = 0
    w = (serS.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfS.iterrows():
    clf = 0
    for p in range(0, len(serS)):
        if row['DATA_TYPE'] == serS.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfS['RANGE_S'] = clf 

In [9]:
serSI = dfSI.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serSI)):
    arrlimt = []
    ml = 0
    w = (serSI.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfSI.iterrows():
    clf = 0
    for p in range(0, len(serSI)):
        if row['DATA_TYPE'] == serSI.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfSI['RANGE_SI'] = clf 

In [10]:
dfS.to_csv("dfS_tot.csv", sep=';')
dfSI.to_csv("dfSI_tot.csv", sep=';')

In [11]:
dfS.to_csv("dfS.csv", sep=';')
dfSI.to_csv("dfSI.csv", sep=';')